In [1]:
import numpy as np
import scipy.linalg as la
import pickle

from scipy.optimize import minimize
from itertools import product

from qulacs.gate import CNOT, RX, RY, RZ
from qulacsvis import circuit_drawer

from qiskit import *
from qiskit.quantum_info.states.random import random_density_matrix
from qiskit.quantum_info import Statevector, DensityMatrix, partial_trace

import matplotlib.pyplot as plt
from mpl_axes_aligner import shift
plt.rcParams.update(
    {"text.usetex": True, "font.family": "serif", "font.size": 10}
)

In [2]:
def dictionary_of_actions(num_qubits):
    dictionary = dict()
    i = 0
    for c, x in product(range(num_qubits),
                        range(1, num_qubits)):
        dictionary[i] =  [c, x, num_qubits, 0]
        i += 1
    for r, h in product(range(num_qubits),
                           range(1, 4)):
        dictionary[i] = [num_qubits, 0, r, h]
        i += 1
    return dictionary

def make_circuit_qiskit(action, qubits):
    ctrl = action[0]
    targ = (action[0] + action[1]) % qubits
    rot_qubit = action[2]
    rot_axis = action[3]
#     print(1)
    if ctrl < qubits:
        circuit.cx([ctrl], [targ])
    if rot_qubit < qubits:
        if rot_axis == 1:
            circuit.rx(0, rot_qubit) # TODO: make a function and take angles
        elif rot_axis == 2:
            circuit.ry(0, rot_qubit)
        elif rot_axis == 3:
            circuit.rz(0, rot_qubit)
        else:
            stop
    return circuit

In [3]:
qubits, rank, seed, seed_agent = 2,4,5,100

In [4]:
data = np.load(f'results/global_COBYLA/h_s_{qubits}_rank_{rank}_{seed}/summary_100.npy',allow_pickle=True)[()]
# data = np.load(f'results/global_COBYLA/h_s_{qubits}_reduce_heisen/summary_{seed_agent}.npy',allow_pickle=True)[()]

FileNotFoundError: [Errno 2] No such file or directory: 'results/global_COBYLA/h_s_2_rank_4_5/summary_100.npy'

In [ ]:
episodes = 10000
test_train = 'train'
success_ep, success_ep_error = [], []
average_err = 0
for ep_no in range(episodes):
    data_error_last = data[test_train][ep_no]['errors'][-1]
    if data_error_last <= 1e-5:
        success_ep.append(ep_no)
        success_ep_error.append(data_error_last)
        average_err += data_error_last
average_err /= len(success_ep)
print(f'First successful episode: {success_ep[0]} and error {round(success_ep_error[0],6)}, total successful episode: {len(success_ep)}')


x = [data[test_train][ep_no]['actions'] for ep_no in success_ep]
circuit_info_dict = {}
depth_list = []
oneq_gate_list = []
twoq_gate_list = []
gate_num_list = []
for succ_ep in success_ep:
    actions = data[test_train][succ_ep]['actions']
    circuit = QuantumCircuit(qubits)
    for a in actions:
        action = dictionary_of_actions(qubits)[a]
        final_circuit = make_circuit_qiskit(action, qubits)
    gate_info = final_circuit.count_ops()
    key_list = gate_info.keys()
    one_gate, two_gate = 0,0
    for k in key_list:
        if k == 'cx':
            two_gate += gate_info[k]
        else:
            one_gate += gate_info[k]
    twoq_gate_list.append(two_gate)
    oneq_gate_list.append(one_gate)
    gate_num_list.append(one_gate+two_gate)        
    depth_list.append(final_circuit.depth())
    
print(f'The average depth is: {np.mean(depth_list)}, minimum depth: {min(depth_list)}, 1st-depth: {depth_list[0]}, 100-depth: {np.mean(depth_list[0:100])}')
print(f'The average gate count: ROT={np.mean(oneq_gate_list)},CX={np.mean(twoq_gate_list)}')
print(f'The min gate count: ROT={oneq_gate_list[gate_num_list.index(min(gate_num_list))]},CX={twoq_gate_list[gate_num_list.index(min(gate_num_list))]}')
success_ep_error[gate_num_list.index(min(gate_num_list))]

In [ ]:
succ_ep = success_ep[gate_num_list.index(min(gate_num_list))]
thetas = data['train'][succ_ep]['opt_ang']
thetas = list(thetas[-1])
total_gate = oneq_gate_list[gate_num_list.index(min(gate_num_list))] + \
                            twoq_gate_list[gate_num_list.index(min(gate_num_list))]
rot_count = oneq_gate_list[gate_num_list.index(min(gate_num_list))]

In [ ]:
state_to_diag_gen = random_density_matrix(2**qubits, rank=rank, seed=seed)
final_circuit = QuantumCircuit.from_qasm_str(data['train'][succ_ep]['save_circ'][-1])
final_circuit.data = final_circuit[:total_gate]

In [ ]:
def fix_structure_agent_ansatz(theta):
    assert len(theta) == rot_count, f"there are {rot_count} rotations!"
    ansatz = QuantumCircuit(qubits)
    rot_idx = 0
    for i in final_circuit.data:
        if i[0].name == 'cx':
            ansatz.cx(i[1][0].index, i[1][1].index)
        if i[0].params != [0]:
            if i[0].name == 'rx':
                ansatz.rx(theta[rot_idx], i[1][0].index)
                rot_idx +=1
            elif i[0].name == 'ry':
                ansatz.ry(theta[rot_idx], i[1][0].index)
                rot_idx +=1
            elif i[0].name == 'rz':
                ansatz.rz(theta[rot_idx], i[1][0].index)
                rot_idx +=1
    return ansatz

def cost_func(n, theta, dm, trace_before_diag):
    """returns the cost function value"""
    """for particular choice of layered ansatz and angle"""
    dm_evo = dm.evolve(fix_structure_agent_ansatz(theta))
    dm_evo = np.kron(dm_evo.data, dm_evo.data)
    
    qc_cost = QuantumCircuit(2*n)
    for i in range(n):
        qc_cost.cx([i+n], [i])
    dm_evo = DensityMatrix(dm_evo).evolve(qc_cost)
    
    par_trace = partial_trace(dm_evo, range(n))
    trace_after_diag = np.trace(par_trace.data @ par_trace.data).real    
    return trace_before_diag - trace_after_diag

def eigenvals_func(dm, ansatz):
    """Return the eigenvalues"""
#     qc_eigenvals = ansatz
    dm_evo = DensityMatrix(dm).evolve(ansatz)
    dm_evo = dm_evo.data
    diagnls = list((np.diagonal(dm_evo)).real)
    diag_A = []
    diag_A.append(diagnls)
    return diag_A, dm_evo

In [ ]:
qubits, rank, total_seed = 2,4,500
optimization_info = {}
for seed in range(total_seed):
    rdm = random_density_matrix(2**qubits, rank = rank, seed = seed)
    rdm_data = rdm.data
    trace_before_diag = np.trace(np.matmul(rdm_data,rdm_data)).real
    true_eig = list(reversed(np.sort(la.eig(rdm_data)[0].real)))
    print(true_eig)
    def cost(theta):
        return cost_func(qubits, theta, rdm , trace_before_diag)
    hea_data = {}
    layer_list, eig_err_list = [], []
    cost_list, ang_list, nfev_list = [], [], []
    for _ in range(10):
        initial_guess = np.random.random(rot_count)
        res = minimize(cost, initial_guess, method = 'COBYLA', options={'maxiter': 500})
        cost_list.append(res['fun'])
        ang_list.append(res['x'])
        nfev_list.append(res.nfev)
    
        theta = ang_list[cost_list.index(np.min(cost_list))]
        ansatz = fix_structure_agent_ansatz(theta)
        inf_eig = list(reversed(np.sort(eigenvals_func(rdm, ansatz)[0])[0]))
#         print(inf_eig)
        err = 0
        for eigno in range(4):
#             print(inf_eig[eigno], true_eig[eigno])
            err += np.abs(inf_eig[eigno] - true_eig[eigno])**2
        eig_err_list.append(err)
#     print(f'{seed: {error}')
    print(f'seed: {seed}, min, err: {np.min(eig_err_list)}')
    optimization_info[f'{seed}'] = [nfev_list, eig_err_list]
with open(f'plot_data/{qubits}_qubit_mixed_rank_constant_structure_opt_data.p', 'wb') as fp:
        pickle.dump(optimization_info, fp)

In [ ]:
fig, ax = plt.subplots( 1, 1, sharey=True, figsize = (4, 3))
fixed_ansatz_data = []
with (open(f"plot_data/{qubits}_qubit_mixed_rank_constant_structure_opt_data.p", "rb")) as openfile:
    while True:
        try:
            fixed_ansatz_data.append(pickle.load(openfile))
        except EOFError:
            break
optimization_info = fixed_ansatz_data[0]
x = []
for k  in optimization_info.keys():
    x.append(np.min(optimization_info[k][1]))
x1 = list(np.copy(x))
x[x.index(np.min(x))] = 0
ax.semilogy(x, 'bo')
# ax.semilogy([4], [6.9*1e-7], 'ro', label = 'State\'s ansatz')
ax.axhline(y = np.mean(x), color = 'k', label = 'Average error')
ax.set_xlabel('Number of states')
ax.set_ylabel('$\sum_i^{m}(\\lambda_i - \\lambda^{\\prime}_i)^2$')
ax.legend()
fig.tight_layout()
# plt.savefig(f'plot/{qubits}_qubit_fixed_ansatz.pdf')
# plt.savefig(f'plot/{qubits}_qubit_fixed_ansatz.png')

In [ ]:
optimization_info = fixed_ansatz_data[0]
x = []
for k  in optimization_info.keys():
    x.append(np.min(optimization_info[k][1]))
x1 = list(np.copy(x))

In [ ]:
sterr1em5,sterr1em4,sterr1em3,sterr1em2  = [], [],[], []
for x in x1:
    if x <= 1e-4 and x >= 1e-5:
        sterr1em5.append(x)
    elif x <= 1e-3 and x > 1e-4:
        sterr1em4.append(x)
    elif x <= 1e-2 and x > 1e-3:
        sterr1em3.append(x)
    else:
        sterr1em2.append(x)
states_err = [len(sterr1em5), len(sterr1em4), len(sterr1em3), len(sterr1em2)]
# print(sum(states_err))

In [ ]:
fig, ax = plt.subplots(figsize =(7, 3))
ax.barh([1,2,3,4], list(reversed(states_err)))
ax.grid(b = True, color ='grey',
        linestyle ='--', linewidth = 0.5,
        alpha = 0.4)
ax.invert_xaxis()
for i in ax.patches:
    plt.text(i.get_width()+0.35, i.get_y()+0.3,
             str(round((i.get_width()), 2)),
             fontsize = 12, fontweight ='bold',
             color ='black')

ax.set_yticks(list(reversed(range(1, 5))), ('$10^{-5}\\leq\\Delta < 10^{-4}$', '$10^{-4}\\leq\\Delta < 10^{-3}$', '$10^{-3}\\leq\\Delta < 10^{-2}$', '$10^{-2}\\leq\\Delta < 10^{-1}$'))
ax.invert_xaxis()
# for l in labels:
ax.set_xticks([])
fig.tight_layout()
plt.savefig('plot/2_qubit_fixed_ansatz.pdf')
plt.savefig('plot/2_qubit_fixed_ansatz.png')
# ax.set_yticks([1e-5, 1e-4])